<a href="https://colab.research.google.com/github/alikhosravi/Spatial-Accessibility/blob/main/Spatial%20accessibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌏 Install packages




In [ ]:
!pip install geopandas
!pip install mapclassify
!pip install fitter
!pip install libpysal

# ⭐ Import libraries

In [ ]:
import seaborn as sns
import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
import pandas
from google.colab import drive
import numpy as np
from scipy import stats
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import robust_scale, minmax_scale
from libpysal.weights import KNN
import matplotlib
import math
from sklearn.cluster import KMeans
from scipy.stats import entropy
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
drive.mount('/content/drive')
db = gpd.read_file('/content/drive/My Drive/Isfahan/Inputs/MyBlocks.gpkg')
variables=['T2H', 'T2L', 'T2C', 'T2P']

# ⚖ Remove Outliers

In [ ]:
Opimaum_Zscore=dict()
for var in variables:
  df = pandas.DataFrame(list(db[var]), columns=["Data"]) 
  for col in df.columns:
      col_zscore = col + "_zscore"
      df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)

  def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.2, max_hist = None, bins = 50):
      fig = plt.figure(figsize=(20, 6))
      fig.suptitle(col, fontsize=16)
      plt.subplot(1,3,1)
      if max_hist == None:
          sns.histplot(df[col], kde=False, bins = 50,color="r")
      else :
          sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
      plt.subplot(1,3,2)
      sns.boxplot(df[col])
      plt.subplot(1,3,3)
      z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
      global Oz
      Oz= z_score_inspect[2]
      return Oz
      plt.show()

  def outlier_zscore(df, col, min_z=1, max_z = 5, step = 0.1, print_list = False):
      z_scores = df["Data_zscore"]
      threshold_list = []
      for threshold in np.arange(min_z, max_z, step):
          threshold_list.append((threshold, len(np.where(z_scores > threshold)[0])))
          df_outlier = pandas.DataFrame(threshold_list, columns = ['threshold', 'outlier_count'])
          df_outlier['pct'] = (df_outlier.outlier_count - df_outlier.outlier_count.shift(-1))/df_outlier.outlier_count*100
      plt.plot(df_outlier.threshold, df_outlier.outlier_count)
      best_treshold = round(df_outlier.iloc[df_outlier.pct.argmax(), 0],2)
      outlier_limit = int(df[col].dropna().mean() + (df[col].dropna().std()) * df_outlier.iloc[df_outlier.pct.argmax(), 0])
      percentile_threshold = stats.percentileofscore(df[col].dropna(), outlier_limit)
      plt.vlines(best_treshold, 0, df_outlier.outlier_count.max(), 
                colors="r", ls = ":"
                )
      plt.annotate("Zscore : {}\nValue : {}\nPercentile : {}".format(best_treshold, outlier_limit, 
                                                                    (np.round(percentile_threshold, 3), 
                                                                      np.round(100-percentile_threshold, 3))), 
                  (best_treshold, df_outlier.outlier_count.max()/2))
      #plt.show()
      if print_list:
          print(df_outlier)
      return (plt, df_outlier, best_treshold, outlier_limit, percentile_threshold)
  outlier_inspect(df,"Data")
  Opimaum_Zscore[var] = Oz

for variable in variables:
  db=db[abs(stats.zscore(db[variable]))<Opimaum_Zscore[variable]]

# ✅ Power Transformation

In [ ]:
Transformed_db = db.loc[:,tuple(variables)]
for variable in variables:
  data = list(db[variable]);
  data = np.array(data);
  pt = PowerTransformer()
  pt.fit(data.reshape(-1,1));
  mm = MinMaxScaler()
  mm.fit(pt.transform(data.reshape(-1,1)))
  Transformed_db[variable] = 1-mm.transform(pt.transform(data.reshape(-1,1)))

# ⚡ Entropy weighting

In [ ]:
def Entropy(labels, base=len(Transformed_db)):
  value,counts = np.unique(labels, return_counts=True)
  return entropy(counts, base=base)
entropies=dict()
for variable in Transformed_db[variables]:
  entropies[variable]=1-Entropy(Transformed_db[variable])
weights=dict()
for variable in entropies:
  weights[variable] = entropies[variable]/sum(entropies.values())

# 💢 Clustering


In [ ]:
def Entropy(labels, base=len(Transformed_db)):
  value,counts = np.unique(labels, return_counts=True)
  return entropy(counts, base=base)
entropies=dict()
for variable in Transformed_db[variables]:
  entropies[variable]=1-Entropy(Transformed_db[variable])
weights=dict()
for variable in entropies:
  weights[variable] = entropies[variable]/sum(entropies.values())

Weighted_db = pandas.DataFrame()
for variable in variables:
  Weighted_db[variable] = Transformed_db[variable]*weights[variable]

def Att(method):
  global Attributes
  Means_sum = [0]*5
  for variable in variables:
    Means_sum=[int(x + y) for x, y in zip(Means_sum, list(db.groupby(method)[variable].mean()))]
  sorted_clusters=[]
  for dis in Means_sum:
    sorted_clusters.append(sorted(Means_sum, reverse= True).index(dis))
  return sorted_clusters

kmeans = KMeans(n_clusters=5)
AP = np.array(AHP);
k5cls = kmeans.fit(AP.reshape(-1,1))
db['KNN'] = k5cls.labels_

atr=Att('KNN')
def adjust(r , sr):
  return sr[r]
db['KNN'] = db.apply(lambda row : adjust(row['KNN'], atr), axis = 1)


from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(n_clusters=5)
yhat = model.fit(AP.reshape(-1,1))
db['AGG'] = yhat.labels_
atr2=Att('AGG')
def adjust(r , sr):
  return sr[r]
db['AGG'] = db.apply(lambda row : adjust(row['AGG'], atr2), axis = 1)

bis = pandas.read_csv('bisect.csv')
db['BIS']=list(bis['0'])
atr2=Att('BIS')
def adjust(r , sr):
  return sr[r]
db['BIS'] = db.apply(lambda row : adjust(row['BIS'], atr2), axis = 1)

# 🥇 Supervised Learning

In [ ]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


In [ ]:
Superdb=db.loc[:,['D2H', 'D2L', 'D2C', 'D2P', 'Diff','geometry']]
Superdb= Superdb.loc[Superdb['Diff'] != 'Different values']
def stringer(x):
  return str(x)
Superdb['Diff'] = Superdb.apply(lambda row : stringer(row['Diff']), axis = 1)


In [ ]:

# Split-out validation dataset
array = DF.values
X = array[:,0:4]
y = array[:,4]
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
#Superdb = Superdb.drop(columns=['geometry'])
grouped = Superdb.groupby(Superdb['Diff'])

DF = pandas.DataFrame()
for cl in list(Superdb['Diff'].unique()):
  df_new = grouped.get_group(cl)
  df_new = df_new.iloc[:Superdb.groupby('Diff').size().min(),:]
  DF = DF.append(df_new)

X = DF
array = X.values

X = array[:,0:4]
y = array[:,4]

X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

# Compare Algorithms
pyplot.boxplot(results, labels=names)
pyplot.title('Algorithm Comparison')
pyplot.show()